In [4]:
import cudaq
from typing import List
import numpy as np 
from cudaq import spin
from scipy.optimize import minimize
from scipy.special import softmax
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, auc

cudaq.set_target("nvidia-mqpu")
target = cudaq.get_target()
num_qpus = target.num_qpus()


df_input = np.loadtxt("qc_pca_train_input_bace_12q2")
df_classes = np.loadtxt("qc_pca_train_classes_bace_12q2") - 1

df_test_input = np.loadtxt("qc_pca_test_input_bace_12q2")
df_test_classes = np.loadtxt("qc_pca_test_classes_bace_12q2") - 1

m = df_classes.size  #Number of data samples 
num_classes = int(max(df_classes)) + 1 #Number of classes
num_qubits = df_input.shape[1]
ansatz_layers = 2
maxiter = 10
num_theta = num_qubits * ansatz_layers
theta_values = np.zeros(num_theta)
# theta_values = np.array([-0.879411278071147, -2.163050261905418])
data_values  = df_input
class_values = df_classes

def gen_default_prob_class(num_qubits, num_classes):
    prob_class = np.zeros(2**num_qubits)
    for i in range(2**num_qubits):
        prob_class[i] = i % num_classes
    return prob_class

prob_class = gen_default_prob_class(num_qubits, num_classes)

In [5]:
@cudaq.kernel
def kernel(data: np.ndarray, theta: np.ndarray):
    
    q = cudaq.qvector(num_qubits)
    
    #Data encoding
    for i in range(num_qubits):
        ry(data[i], q[num_qubits-i-1])
    for i in range(num_qubits - 1):
        x.ctrl(q[i + 1], q[i])

    #Parameterised ansatz
    for i in range(ansatz_layers):
        for j in range(num_qubits):
            ry(theta[i * num_qubits + j], q[num_qubits-j-1])
        for j in range(num_qubits - 1):
            x.ctrl(q[j + 1], q[j])

def cost_function(theta_values):
    cost = 0.0
    
    qpu_counter = 0 
    for i in range(m):
        c = 0.0
        
        sv = np.array(cudaq.get_state_async(kernel, data_values[i], theta_values), qpu_id = i)
        
        if qpu_counter%num_qpus == 0: 
            qpu_counter = 0 

        prob = softmax(np.real(np.abs(sv) ** 2))
        for j in range(2**num_qubits):
            if prob_class[j] == class_values[i]:
                c += prob[j]
        cost += -np.log(c)
    cost = cost / m
    return cost

def find_prob(data, theta, classes):
    p = np.zeros(len(data))
    for i in range(len(data)):
        c = 0.0
        sv = np.array(cudaq.get_state(kernel, data[i], theta))
        prob = np.real(np.abs(sv) ** 2)
        for j in range(2**num_qubits):
            if prob_class[j] == classes[i]:
                c += prob[j]
        p[i] = c
    return p

In [6]:

m = 10000
data_values = np.random.rand(m, 2)

In [7]:
results = []
for i in range(m): 
    
    results.append(cudaq.get_state_async(kernel, data_values[i], theta_values, qpu_id = 0))

svs = []
for i in range(len(results)): 
    svs.append(np.array(results[i].get()))


In [8]:
results = []

qpu_counter = 0 

for i in range(m): 
    
    results.append(cudaq.get_state_async(kernel, data_values[i], theta_values, qpu_id = qpu_counter))
    
    qpu_counter += 1 

    if qpu_counter%num_qpus == 0: 
        qpu_counter = 0 

svs = []
for i in range(len(results)): 
    svs.append(np.array(results[i].get()))
    

In [9]:
loss = []

loss.append(cost_function(theta_values))


def callback(xk):
    result = cost_function(xk)
    print(result, xk)
    loss.append(result)
    
    
result = minimize(cost_function, theta_values, method='COBYLA', callback=callback, options={'maxiter': maxiter})

TypeError: array() got an unexpected keyword argument 'qpu_id'

In [ ]:
plt.plot(loss)

In [ ]:
t_final = result.x
p_final = find_prob(data_values,t_final, class_values)
print(p_final)

In [ ]:
print(loss[1:])

In [ ]:
#Use only for binary classification
p_refined = np.zeros(m)
for i in range(m):
    if class_values[i] == 1:
        p_refined[i] = p_final[i]
    else:
        p_refined[i] = 1 - p_final[i]
roc_auc_score(class_values, p_refined)

In [ ]:
# Compute ROC curve
fpr, tpr, thresholds = roc_curve(class_values, p_refined)

# Compute ROC AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
p_test_final = find_prob(df_test_input,t_final, df_test_classes)
#Use only for binary classification
p_test_refined = np.zeros(len(df_test_classes))
for i in range(len(df_test_classes)):
    if df_test_classes[i] == 1:
        p_test_refined[i] = p_final[i]
    else:
        p_test_refined[i] = 1 - p_test_final[i]
roc_auc_score(df_test_classes, p_test_refined)